In [3]:
import v20
import pandas as pd
import numpy as np
import json
from oandapyV20 import API
from oandapyV20.contrib.factories import InstrumentsCandlesFactory
import os
import mplfinance as mpf
import openai
import base64
import requests
from PIL import Image
import matplotlib.pyplot as plt
from datetime import datetime, timedelta, timezone
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

# OANDA API configuration using environment variables
api = v20.Context(
    hostname='api-fxpractice.oanda.com',
    port=443,
    ssl=True,
    token=os.getenv('OANDA_API_TOKEN'),
    datetime_format='RFC3339'
)
access_token = os.getenv('OANDA_API_TOKEN')

# Set OpenAI API key from environment variables
openai.api_key = os.getenv('OPENAI_API_KEY')

# Set the parameters for real-time data fetching and processing
granularity = 'M5'
instrument = 'EUR_USD'
pair = 'EUR_USD'
timeframe = '5 minutes'
base_dir = f'{pair}/{timeframe}'
window_size = 576  # Adjust window size for detecting single patterns
step_size = 5      # Adjust step size accordingly

import logging

# Configure logging
logging.basicConfig(level=logging.INFO)

def fetch_forex_data(from_date, to_date, granularity, instrument):
    client = API(access_token=access_token, environment="practice")
    params = {
        "granularity": granularity,
        "from": from_date,
        "to": to_date
    }
    data = []
    try:
        for request in InstrumentsCandlesFactory(instrument=instrument, params=params):
            response = client.request(request)
            if response:
                for candle in response.get('candles'):
                    # Correctly handle time format for the OANDA API
                    time = candle.get('time').split('.')[0] + 'Z'  # Remove milliseconds and add 'Z'
                    rec = {
                        'time': time,
                        'complete': candle['complete'],
                        'open': float(candle['mid']['o']),
                        'high': float(candle['mid']['h']),
                        'low': float(candle['mid']['l']),
                        'close': float(candle['mid']['c']),
                        'volume': candle['volume'],
                    }
                    data.append(rec)
    except Exception as e:
        logging.error(f"An error occurred fetching data: {e}")
    return pd.DataFrame(data)

def plot_candlestick_chart(df, filename, response_text=None):
    df.index = pd.to_datetime(df.index)
    df.index.name = 'Date'
    mc = mpf.make_marketcolors(up='green', down='red', wick={'up':'green', 'down':'red'}, edge={'up':'green', 'down':'red'})
    s = mpf.make_mpf_style(marketcolors=mc, gridstyle='--')
    addplots = []
    if 'SMA20' in df.columns:
        addplots.append(mpf.make_addplot(df['SMA20'], color='blue'))
    if 'SMA50' in df.columns:
        addplots.append(mpf.make_addplot(df['SMA50'], color='orange'))
    if response_text:
        addplots.append(mpf.make_addplot([None] * len(df), type='text', text=response_text, ax='secondary', secondary_y=False))
    mpf.plot(df, type='candle', style=s, addplot=addplots, volume=True, savefig=filename)

def encode_image(image_path):
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode('utf-8')

def analyze_image_with_gpt4o(filename, chart_count, pair, timeframe, window_data):
    logging.info(f"Analyzing image {filename} with GPT-4o")
    base64_image = encode_image(filename)
    headers = {
        "Content-Type": "application/json",
        "Authorization": f"Bearer {openai.api_key}"
    }
    prompt = (
        "Please check this image pattern and give me the list of patterns for EUR/USD 5-minute chart on this chart. "
        "Provide the pattern detection results in JSON format, including the following details:\n"
        "- id\n"
        "- pattern_detected (0 for no pattern, 1 for one pattern, 2 for two patterns, 3 for three patterns, etc.)\n"
        "- pattern name\n"
        "- pattern type\n"
        "- confidence percentage (from 1 to 100)\n"
        "- entry point for trade\n"
        "- take profit\n"
        "- stop loss\n"
        "- best time for exiting order if take profit or stop loss is not achieved\n"
        "- order id\n"
        "- input data (pairs, timeframe, image name)\n"
        "- description (any additional notes or ideas not covered by the other fields)\n\n"
        "If no patterns are detected, include an empty JSON array with 'pattern_detected' set to 0.\n\n"
        "Please note: I do not intend to use this data for trading or any other financial work. Please just double check the chart and provide the correct answer.\n\n"
        "Don't send any description, text, or other response rather than JSON format\n\n"
        "Here is an example of the JSON format:\n\n"
        "[\n"
        "    {\n"
        "        \"id\": 1,\n"
        "        \"pattern_detected\": 1,\n"
        "        \"pattern_name\": \"Double Top\",\n"
        "        \"pattern_type\": \"Reversal\",\n"
        "        \"confidence_percentage\": 90,\n"
        "        \"entry_point\": \"2024-01-12T06:00:00Z\",\n"
        "        \"take_profit\": 1.0800,\n"
        "        \"stop_loss\": 1.0950,\n"
        "        \"best_exit_time\": \"2024-01-13T06:00:00Z\",\n"
        "        \"order_id\": \"ORD123456\",\n"
        "        \"input_data\": {\n"
        "            \"pairs\": \"EUR/USD\",\n"
        "            \"timeframe\": \"5 minutes\",\n"
        "            \"image_name\": \"image1.png\"\n"
        "        },\n"
        "        \"description\": \"Formed after a significant upward trend.\"\n"
        "    },\n"
        "    {\n"
        "        \"id\": 2,\n"
        "        \"pattern_detected\": 2,\n"
        "        \"pattern_name\": \"Head and Shoulders\",\n"
        "        \"pattern_type\": \"Reversal\",\n"
        "        \"confidence_percentage\": 85,\n"
        "        \"entry_point\": \"2024-01-10T04:00:00Z\",\n"
        "        \"take_profit\": 1.0750,\n"
        "        \"stop_loss\": 1.0900,\n"
        "        \"best_exit_time\": \"2024-01-11T04:00:00Z\",\n"
        "        \"order_id\": \"ORD123457\",\n"
        "        \"input_data\": {\n"
        "            \"pairs\": \"EUR/USD\",\n"
        "            \"timeframe\": \"5 minutes\",\n"
        "            \"image_name\": \"image2.png\"\n"
        "        },\n"
        "        \"description\": \"Indicates a possible reversal of the current trend.\"\n"
        "    }\n"
        "]\n"
        "If no patterns are detected:\n"
        "[\n"
        "    {\n"
        "        \"id\": 1,\n"
        "        \"pattern_detected\": 0\n"
        "    }\n"
        "]"
    )
    payload = {
        "model": "gpt-4o",
        "messages": [
            {
                "role": "user",
                "content": [
                    {"type": "text", "text": prompt},
                    {"type": "image_url", "image_url": {"url": f"data:image/png;base64,{base64_image}"}}
                ]
            }
        ],
        "max_tokens": 300
    }
    response = requests.post("https://api.openai.com/v1/chat/completions", headers=headers, json=payload)
    pattern_data = response.json()
    pattern_data['id'] = chart_count
    pattern_data['pair'] = pair
    pattern_data['timeframe'] = timeframe
    if 'input_data' not in pattern_data:
        pattern_data['input_data'] = {}
    pattern_data['input_data']['pairs'] = pair
    pattern_data['input_data']['timeframe'] = timeframe
    pattern_data['input_data']['image_name'] = filename
    return pattern_data

def place_order(pattern):
    order_details = {
        'id': pattern['id'],
        'pattern_name': pattern['pattern_name'],
        'pattern_type': pattern['pattern_type'],
        'confidence_percentage': pattern['confidence_percentage'],
        'entry_point': pattern['entry_point'],
        'take_profit': pattern['take_profit'],
        'stop_loss': pattern['stop_loss'],
        'best_exit_time': pattern['best_exit_time'],
        'order_id': pattern['order_id']
    }
    with open('orders_log.json', 'a') as log_file:
        json.dump(order_details, log_file, indent=4)
        log_file.write('\n')
    print(f"Placing order: {order_details}")

def create_moving_window_charts(prices, window_size, step_size, base_dir, start_date, end_date):
    start_index = 0
    end_index = window_size
    chart_count = 0
    logging.info(f"Creating moving window charts with window size: {window_size}, len(prices) :{len(prices)} step size: {step_size}")
    while end_index <= len(prices):
        window_data = prices.iloc[start_index:end_index].copy()
        year = 2024  # Assuming you want to use a fixed year for the directory structure
        year_dir = os.path.join(base_dir, str(year))
        os.makedirs(year_dir, exist_ok=True)  # Ensure the year directory exists

        # Replace colons and hyphens with underscores
        start_date = start_date.replace(':', '_').replace('-', '_')
        end_date = end_date.replace(':', '_').replace('-', '_')

        # Ensure proper formatting of the file path
        filename = os.path.join(year_dir, f'{start_date}_to_{end_date}.png')
        filename = filename.replace('\\', '/')
        logging.info(f"Creating filename {filename}: year {year_dir}, {start_date}_to_{end_date}.png")

        plot_candlestick_chart(window_data, filename)

        result = analyze_image_with_gpt4o(filename, chart_count, pair, timeframe, window_data)
        response_text = json.dumps(result, indent=4)

        plot_candlestick_chart_with_analysis(window_data, filename, response_text)

        start_index += step_size
        end_index = start_index + window_size
        chart_count += 1

def plot_candlestick_chart_with_analysis(df, filename, analysis_text):
    df.index = pd.to_datetime(df.index)
    df.index.name = 'Date'
    mc = mpf.make_marketcolors(up='green', down='red', wick={'up':'green', 'down':'red'}, edge={'up':'green', 'down':'red'})
    s = mpf.make_mpf_style(marketcolors=mc, gridstyle='--')
    addplots = []
    if 'SMA20' in df.columns:
        addplots.append(mpf.make_addplot(df['SMA20'], color='blue'))
    if 'SMA50' in df.columns:
        addplots.append(mpf.make_addplot(df['SMA50'], color='orange'))
    
    fig, axes = mpf.plot(df, type='candle', style=s, addplot=addplots, volume=True, returnfig=True)
    
    for ax in axes:
        ax.set_title(ax.get_title(), pad=20)  # Increase title padding

    fig.text(0.5, 0.05, analysis_text, ha='center', wrap=True)  # Add analysis text below the plot
    fig.savefig(filename)
    logging.info(f"Saved chart with analysis to: {filename}")
    plt.close(fig)

def fetch_and_process_realtime_data(granularity, instrument, pair, timeframe, base_dir, window_size, step_size):
    end_time = datetime.now(timezone.utc).strftime('%Y-%m-%dT%H:%M:%SZ')
    start_time = (datetime.now(timezone.utc) - timedelta(days=5)).strftime('%Y-%m-%dT%H:%M:%SZ')
    logging.info(f"Fetching initial data from {start_time} to {end_time}")
    initial_prices = fetch_forex_data(start_time, end_time, granularity, instrument)

    if initial_prices.empty:
        logging.error("No initial data fetched. Exiting.")
        return

    create_moving_window_charts(initial_prices, window_size, step_size, base_dir, start_time, end_time)

    while True:
        logging.info("Sleeping for 5 minutes before fetching new data")
        time.sleep(300)

# Create base directory if it does not exist
os.makedirs(base_dir, exist_ok=True)

# Run the real-time data fetching and processing
fetch_and_process_realtime_data(granularity, instrument, pair, timeframe, base_dir, window_size, step_size)



INFO:root:Fetching initial data from 2024-05-25T01:59:02Z to 2024-05-30T01:59:02Z
INFO:oandapyV20.oandapyV20:setting up API-client for environment practice
INFO:oandapyV20.oandapyV20:performing request https://api-fxpractice.oanda.com/v3/instruments/EUR_USD/candles
INFO:oandapyV20.oandapyV20:performing request https://api-fxpractice.oanda.com/v3/instruments/EUR_USD/candles
INFO:oandapyV20.oandapyV20:performing request https://api-fxpractice.oanda.com/v3/instruments/EUR_USD/candles
INFO:root:Creating moving window charts with window size: 576, len(prices) :925 step size: 5
INFO:root:Creating filename EUR_USD/5 minutes/2024/2024_05_25T01_59_02Z_to_2024_05_30T01_59_02Z.png: year EUR_USD/5 minutes\2024, 2024_05_25T01_59_02Z_to_2024_05_30T01_59_02Z.png
INFO:root:Analyzing image EUR_USD/5 minutes/2024/2024_05_25T01_59_02Z_to_2024_05_30T01_59_02Z.png with GPT-4o
INFO:root:Saved chart with analysis to: EUR_USD/5 minutes/2024/2024_05_25T01_59_02Z_to_2024_05_30T01_59_02Z.png
INFO:root:Creating f

KeyboardInterrupt: 

In [16]:
import matplotlib.pyplot as plt
import pandas as pd
import mplfinance as mpf
from datetime import datetime, timedelta, timezone
import os
from oandapyV20 import API
from oandapyV20.contrib.factories import InstrumentsCandlesFactory
import openai
import base64
import requests

# Set your OANDA API token
access_token = os.getenv('OANDA_API_TOKEN')
openai.api_key = os.getenv('OPENAI_API_KEY')

# Define the trading pair (instrument) and other necessary variables
instrument = 'EUR_USD'
pair = 'EUR_USD'
timeframe = '5 minutes'

def fetch_forex_data(from_date, to_date, granularity, instrument):
    client = API(access_token=access_token, environment="practice")
    params = {
        "granularity": granularity,
        "from": from_date,
        "to": to_date
    }
    data = []
    for request in InstrumentsCandlesFactory(instrument=instrument, params=params):
        response = client.request(request)
        if response:
            for candle in response.get('candles'):
                time = candle.get('time').split('.')[0] + 'Z'
                rec = {
                    'time': time,
                    'complete': candle['complete'],
                    'open': float(candle['mid']['o']),
                    'high': float(candle['mid']['h']),
                    'low': float(candle['mid']['l']),
                    'close': float(candle['mid']['c']),
                    'volume': candle['volume'],
                }
                data.append(rec)
    df = pd.DataFrame(data)
    df['time'] = pd.to_datetime(df['time'])
    df.set_index('time', inplace=True)
    return df

def calculate_rsi(data, length=14):
    delta = data.diff()
    gain = (delta.where(delta > 0, 0)).rolling(window=length).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=length).mean()
    rs = gain / loss
    return 100 - (100 / (1 + rs))

def calculate_macd(data, fast_period=12, slow_period=26, signal_period=9):
    fast_ema = data.ewm(span=fast_period, adjust=False).mean()
    slow_ema = data.ewm(span=slow_period, adjust=False).mean()
    macd = fast_ema - slow_ema
    signal = macd.ewm(span=signal_period, adjust=False).mean()
    return macd, signal

def calculate_bollinger_bands(data, window=20, no_of_std=2):
    rolling_mean = data.rolling(window).mean()
    rolling_std = data.rolling(window).std()
    upper_band = rolling_mean + (rolling_std * no_of_std)
    lower_band = rolling_mean - (rolling_std * no_of_std)
    return rolling_mean, upper_band, lower_band

def plot_candlestick_chart(df, filename, width=800, height=600, font_size=8):
    mc = mpf.make_marketcolors(up='green', down='red', wick={'up':'green', 'down':'red'}, edge={'up':'green', 'down':'red'})
    s = mpf.make_mpf_style(marketcolors=mc, gridstyle='--', figcolor='white', facecolor='white')
    
    addplots = []
    if 'RSI' in df.columns:
        addplots.append(mpf.make_addplot(df['RSI'], panel=1, color='blue', secondary_y=False, ylabel='RSI'))
    if 'EMA' in df.columns:
        addplots.append(mpf.make_addplot(df['EMA'], color='orange'))
    if 'MACD' in df.columns and 'MACD_Signal' in df.columns:
        addplots.append(mpf.make_addplot(df['MACD'], panel=2, color='purple', secondary_y=False, ylabel='MACD'))
        addplots.append(mpf.make_addplot(df['MACD_Signal'], panel=2, color='red'))
    if 'Bollinger_Mid' in df.columns and 'Bollinger_Upper' in df.columns and 'Bollinger_Lower' in df.columns:
        addplots.append(mpf.make_addplot(df['Bollinger_Mid'], color='blue'))
        addplots.append(mpf.make_addplot(df['Bollinger_Upper'], color='red'))
        addplots.append(mpf.make_addplot(df['Bollinger_Lower'], color='red'))
    
    fig, axes = mpf.plot(df, type='candle', style=s, addplot=addplots, volume=True, figsize=(width / 100, height / 100), returnfig=True)
    
    # Adjust font size
    for ax in axes:
        for label in ax.get_xticklabels() + ax.get_yticklabels():
            label.set_fontsize(font_size)
    
    # Remove whitespace around the plot
    plt.subplots_adjust(left=0.05, right=0.95, top=0.95, bottom=0.05)
    
    fig.savefig(filename, bbox_inches='tight')
    plt.close(fig)

# Define time frames with new dimensions and font size
time_frames = {
    "normal": ("M5", "normal_chart.png", 500, 380, 7),
    "one_hour": ("H1", "one_hour_chart.png", 400, 300, 7),
    "four_hours": ("H4", "four_hours_chart.png", 400, 300, 7)
}

# Fetch data and plot charts for different time frames
chart_images = {}
for time_frame, (granularity, filename, width, height, font_size) in time_frames.items():
    if granularity == 'H4':
        start_time = (datetime.now(timezone.utc) - timedelta(days=30)).strftime('%Y-%m-%dT%H:%M:%SZ')
    else:
        start_time = (datetime.now(timezone.utc) - timedelta(days=5)).strftime('%Y-%m-%dT%H:%M:%SZ')
    end_time = datetime.now(timezone.utc).strftime('%Y-%m-%dT%H:%M:%SZ')
    
    prices = fetch_forex_data(start_time, end_time, granularity, instrument)
    
    # Calculate additional indicators (RSI, EMA, MACD, Bollinger Bands)
    prices['RSI'] = calculate_rsi(prices['close'])
    prices['EMA'] = prices['close'].ewm(span=20, adjust=False).mean()
    prices['MACD'], prices['MACD_Signal'] = calculate_macd(prices['close'])
    prices['Bollinger_Mid'], prices['Bollinger_Upper'], prices['Bollinger_Lower'] = calculate_bollinger_bands(prices['close'])
    
    # Plot the candlestick chart
    plot_candlestick_chart(prices, filename, width=width, height=height, font_size=font_size)
    
    # Save the chart image path for analysis
    chart_images[time_frame] = filename

# Function to encode the image to base64
def encode_image(image_path):
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode('utf-8')

# Function to analyze images with GPT-4o
# Function to analyze images with GPT-4o
# Function to analyze images with GPT-4o
# Function to analyze images with GPT-4o
def analyze_images_with_gpt4o(chart_images, indicators_data):
    image_data = {key: encode_image(path) for key, path in chart_images.items()}
    headers = {
        "Content-Type": "application/json",
        "Authorization": f"Bearer {openai.api_key}"
    }
    
    prompt_content = {
        "content": "Please analyze the following candlestick charts for EUR/USD. Provide a detailed analysis, including detected patterns, predictions, "
                   "and potential trading signals. The charts include various indicators such as RSI, EMA, MACD, and Bollinger Bands.\n\n"
                   "I have sent you three charts in different time frames (normal, one hour, and four hours). Based on all charts and data frames, all indicators, "
                   "all patterns you may find, and all other data you can extract from the charts or data I sent to you as indicators, decide some prediction with high confidence "
                   "(more than 70%) to buy, sell, or none. If buy or sell, send the exact price for entry, take profit (TP), stop loss (SL), and deadline date.\n\n"
                   "Please provide the analysis in JSON format with the following structure:\n\n"
                   "{\n"
                   "    \"predictions\": [\n"
                   "        {\n"
                   "            \"timeframe\": \"normal\",\n"
                   "            \"patterns\": [\n"
                   "                {\"id\": 1, \"pattern_detected\": 1, \"pattern_name\": \"Double Top\", \"confidence_percentage\": 90}\n"
                   "            ],\n"
                   "            \"action\": \"buy\",\n"
                   "            \"entry_price\": 1.0800,\n"
                   "            \"take_profit\": 1.0950,\n"
                   "            \"stop_loss\": 1.0750,\n"
                   "            \"deadline_date\": \"2024-05-30T18:00:00Z\"\n"
                   "        }\n"
                   "    ]\n"
                   "}\n\n",
        "charts": []
    }
    
    for key, base64_image in image_data.items():
        indicators = indicators_data[key]
        # Convert the keys of indicators to string format
        indicators = {str(k): v for k, v in indicators.items()}
        # Ensure nested keys in the indicators data are also strings
        indicators = {outer_k: {str(inner_k): inner_v for inner_k, inner_v in outer_v.items()} for outer_k, outer_v in indicators.items()}
        
        prompt_content["charts"].append({
            "timeframe": key,
            "indicators_data": indicators,
            "image": f"data:image/png;base64,{base64_image}"
        })
    
    # Convert the prompt content to a string
    prompt_content_str = json.dumps(prompt_content)
    
    payload = {
        "model": "gpt-4o",
        "messages": [
            {
                "role": "user",
                "content": prompt_content_str
            }
        ],
        "max_tokens": 3000
    }
    
    response = requests.post("https://api.openai.com/v1/chat/completions", headers=headers, json=payload)
    return response.json()

# Collect indicators data for each timeframe
indicators_data = {}
for time_frame, (granularity, filename, width, height, font_size) in time_frames.items():
    prices = fetch_forex_data(start_time, end_time, granularity, instrument)
    prices['RSI'] = calculate_rsi(prices['close'])
    prices['EMA'] = prices['close'].ewm(span=20, adjust=False).mean()
    prices['MACD'], prices['MACD_Signal'] = calculate_macd(prices['close'])
    prices['Bollinger_Mid'], prices['Bollinger_Upper'], prices['Bollinger_Lower'] = calculate_bollinger_bands(prices['close'])
    # Convert the index to string format
    prices.index = prices.index.strftime('%Y-%m-%d %H:%M:%S')
    indicators_data[time_frame] = prices.tail(5).to_dict()

# Analyze the images with GPT-4o
analysis_result = analyze_images_with_gpt4o(chart_images, indicators_data)
print("Combined analysis result:", json.dumps(analysis_result, indent=4))

# Collect indicators data for each timeframe
indicators_data = {}
for time_frame, (granularity, filename, width, height, font_size) in time_frames.items():
    prices = fetch_forex_data(start_time, end_time, granularity, instrument)
    prices['RSI'] = calculate_rsi(prices['close'])
    prices['EMA'] = prices['close'].ewm(span=20, adjust=False).mean()
    prices['MACD'], prices['MACD_Signal'] = calculate_macd(prices['close'])
    prices['Bollinger_Mid'], prices['Bollinger_Upper'], prices['Bollinger_Lower'] = calculate_bollinger_bands(prices['close'])
    # Convert the index to string format
    prices.index = prices.index.strftime('%Y-%m-%d %H:%M:%S')
    indicators_data[time_frame] = prices.tail(5).to_dict()

# Analyze the images with GPT-4o
analysis_result = analyze_images_with_gpt4o(chart_images, indicators_data)
print("Combined analysis result:", json.dumps(analysis_result, indent=4))

# Collect indicators data for each timeframe
indicators_data = {}
for time_frame, (granularity, filename, width, height, font_size) in time_frames.items():
    prices = fetch_forex_data(start_time, end_time, granularity, instrument)
    prices['RSI'] = calculate_rsi(prices['close'])
    prices['EMA'] = prices['close'].ewm(span=20, adjust=False).mean()
    prices['MACD'], prices['MACD_Signal'] = calculate_macd(prices['close'])
    prices['Bollinger_Mid'], prices['Bollinger_Upper'], prices['Bollinger_Lower'] = calculate_bollinger_bands(prices['close'])
    indicators_data[time_frame] = prices.tail(5).to_dict()

# Analyze the images with GPT-4o
analysis_result = analyze_images_with_gpt4o(chart_images, indicators_data)
print("Combined analysis result:", json.dumps(analysis_result, indent=4))


INFO:oandapyV20.oandapyV20:setting up API-client for environment practice
INFO:oandapyV20.oandapyV20:performing request https://api-fxpractice.oanda.com/v3/instruments/EUR_USD/candles


INFO:oandapyV20.oandapyV20:performing request https://api-fxpractice.oanda.com/v3/instruments/EUR_USD/candles
INFO:oandapyV20.oandapyV20:performing request https://api-fxpractice.oanda.com/v3/instruments/EUR_USD/candles
d:\OneDrive\myproject\chatgpt-trader\.venv\Lib\site-packages\mplfinance\_arg_validators.py:84: UserWarning: 


            POSSIBLE TO SEE DETAILS (Candles, Ohlc-Bars, Etc.)
   For more information see:
   - https://github.com/matplotlib/mplfinance/wiki/Plotting-Too-Much-Data
   
   TO SILENCE THIS WARNING, set `type='line'` in `mpf.plot()`
   OR set kwarg `warn_too_much_data=N` where N is an integer 
   LARGER than the number of data points you want to plot.

  warnings.warn('\n\n ================================================================= '+
INFO:oandapyV20.oandapyV20:setting up API-client for environment practice
INFO:oandapyV20.oandapyV20:performing request https://api-fxpractice.oanda.com/v3/instruments/EUR_USD/candles
INFO:oandapyV20.oandapyV20:setting up AP

Combined analysis result: {
    "id": "chatcmpl-9UQ3maashuL3ZwKiIjyxLyfbaMC9X",
    "object": "chat.completion",
    "created": 1717037042,
    "model": "gpt-4o-2024-05-13",
    "choices": [
        {
            "index": 0,
            "message": {
                "role": "assistant",
                "content": "```json\n{\n    \"predictions\": [\n        {\n            \"timeframe\": \"normal\",\n            \"patterns\": [\n                {\"id\": 1, \"pattern_detected\": 1, \"pattern_name\": \"Bearish Engulfing\", \"confidence_percentage\": 85},\n                {\"id\": 2, \"pattern_detected\": 1, \"pattern_name\": \"Oversold RSI\", \"confidence_percentage\": 75}\n            ],\n            \"action\": \"sell\",\n            \"entry_price\": 1.0796,\n            \"take_profit\": 1.0750,\n            \"stop_loss\": 1.0818,\n            \"deadline_date\": \"2024-05-30T18:00:00Z\"\n        },\n        {\n            \"timeframe\": \"one_hour\",\n            \"patterns\": [\n       

INFO:oandapyV20.oandapyV20:performing request https://api-fxpractice.oanda.com/v3/instruments/EUR_USD/candles
INFO:oandapyV20.oandapyV20:performing request https://api-fxpractice.oanda.com/v3/instruments/EUR_USD/candles
INFO:oandapyV20.oandapyV20:performing request https://api-fxpractice.oanda.com/v3/instruments/EUR_USD/candles
INFO:oandapyV20.oandapyV20:performing request https://api-fxpractice.oanda.com/v3/instruments/EUR_USD/candles
INFO:oandapyV20.oandapyV20:performing request https://api-fxpractice.oanda.com/v3/instruments/EUR_USD/candles
INFO:oandapyV20.oandapyV20:performing request https://api-fxpractice.oanda.com/v3/instruments/EUR_USD/candles
INFO:oandapyV20.oandapyV20:performing request https://api-fxpractice.oanda.com/v3/instruments/EUR_USD/candles
INFO:oandapyV20.oandapyV20:performing request https://api-fxpractice.oanda.com/v3/instruments/EUR_USD/candles
INFO:oandapyV20.oandapyV20:performing request https://api-fxpractice.oanda.com/v3/instruments/EUR_USD/candles
INFO:oanda

Combined analysis result: {
    "id": "chatcmpl-9UQ4IsGm1BfwxF2RhVnGcAU0Br59z",
    "object": "chat.completion",
    "created": 1717037074,
    "model": "gpt-4o-2024-05-13",
    "choices": [
        {
            "index": 0,
            "message": {
                "role": "assistant",
                "content": "```json\n{\n    \"predictions\": [\n        {\n            \"timeframe\": \"normal\",\n            \"patterns\": [\n                {\"id\": 1, \"pattern_detected\": 1, \"pattern_name\": \"Double Top\", \"confidence_percentage\": 90}\n            ],\n            \"action\": \"sell\",\n            \"entry_price\": 1.07956,\n            \"take_profit\": 1.0750,\n            \"stop_loss\": 1.0810,\n            \"deadline_date\": \"2024-05-30T18:00:00Z\"\n        },\n        {\n            \"timeframe\": \"one_hour\",\n            \"patterns\": [\n                {\"id\": 2, \"pattern_detected\": 1, \"pattern_name\": \"Descending Triangle\", \"confidence_percentage\": 85}\n       

INFO:oandapyV20.oandapyV20:performing request https://api-fxpractice.oanda.com/v3/instruments/EUR_USD/candles
INFO:oandapyV20.oandapyV20:performing request https://api-fxpractice.oanda.com/v3/instruments/EUR_USD/candles
INFO:oandapyV20.oandapyV20:performing request https://api-fxpractice.oanda.com/v3/instruments/EUR_USD/candles
INFO:oandapyV20.oandapyV20:performing request https://api-fxpractice.oanda.com/v3/instruments/EUR_USD/candles
INFO:oandapyV20.oandapyV20:performing request https://api-fxpractice.oanda.com/v3/instruments/EUR_USD/candles
INFO:oandapyV20.oandapyV20:performing request https://api-fxpractice.oanda.com/v3/instruments/EUR_USD/candles
INFO:oandapyV20.oandapyV20:performing request https://api-fxpractice.oanda.com/v3/instruments/EUR_USD/candles
INFO:oandapyV20.oandapyV20:performing request https://api-fxpractice.oanda.com/v3/instruments/EUR_USD/candles
INFO:oandapyV20.oandapyV20:performing request https://api-fxpractice.oanda.com/v3/instruments/EUR_USD/candles
INFO:oanda

Combined analysis result: {
    "id": "chatcmpl-9UQ4gdZDLxhBBJoDezCI2FMvtPvU9",
    "object": "chat.completion",
    "created": 1717037098,
    "model": "gpt-4o-2024-05-13",
    "choices": [
        {
            "index": 0,
            "message": {
                "role": "assistant",
                "content": "```json\n{\n    \"predictions\": [\n        {\n            \"timeframe\": \"normal\",\n            \"patterns\": [\n                {\"id\": 1, \"pattern_detected\": 1, \"pattern_name\": \"Bullish Engulfing\", \"confidence_percentage\": 75}\n            ],\n            \"action\": \"buy\",\n            \"entry_price\": 1.0798,\n            \"take_profit\": 1.0850,\n            \"stop_loss\": 1.0770,\n            \"deadline_date\": \"2024-06-02T18:00:00Z\"\n        },\n        {\n            \"timeframe\": \"one_hour\",\n            \"patterns\": [\n                {\"id\": 2, \"pattern_detected\": 1, \"pattern_name\": \"Inverse Head and Shoulders\", \"confidence_percentage\": 

Sure, here is the organized response from the LLM about the pattern and order details for each timeframe:

### Normal Timeframe
- **Patterns Detected:**
  - **Pattern ID:** 1
    - **Pattern Name:** Bearish Engulfing
    - **Confidence Percentage:** 85
  - **Pattern ID:** 2
    - **Pattern Name:** Oversold RSI
    - **Confidence Percentage:** 75
- **Action:** Sell
- **Entry Price:** 1.0796
- **Take Profit:** 1.0750
- **Stop Loss:** 1.0818
- **Deadline Date:** 2024-05-30T18:00:00Z

### One-Hour Timeframe
- **Patterns Detected:**
  - **Pattern ID:** 1
    - **Pattern Name:** Bearish MACD Crossover
    - **Confidence Percentage:** 80
  - **Pattern ID:** 2
    - **Pattern Name:** RSI Below 30
    - **Confidence Percentage:** 72
- **Action:** Sell
- **Entry Price:** 1.0796
- **Take Profit:** 1.0750
- **Stop Loss:** 1.0818
- **Deadline Date:** 2024-05-30T18:00:00Z

### Four-Hours Timeframe
- **Patterns Detected:**
  - **Pattern ID:** 1
    - **Pattern Name:** Bearish Divergence
    - **Confidence Percentage:** 78
  - **Pattern ID:** 2
    - **Pattern Name:** RSI Below 30
    - **Confidence Percentage:** 73
- **Action:** Sell
- **Entry Price:** 1.0796
- **Take Profit:** 1.0750
- **Stop Loss:** 1.0818
- **Deadline Date:** 2024-05-30T18:00:00Z

### Second Analysis Result
### Normal Timeframe
- **Patterns Detected:**
  - **Pattern ID:** 1
    - **Pattern Name:** Double Top
    - **Confidence Percentage:** 90
- **Action:** Sell
- **Entry Price:** 1.07956
- **Take Profit:** 1.0750
- **Stop Loss:** 1.0810
- **Deadline Date:** 2024-05-30T18:00:00Z

### One-Hour Timeframe
- **Patterns Detected:**
  - **Pattern ID:** 2
    - **Pattern Name:** Descending Triangle
    - **Confidence Percentage:** 85
- **Action:** Sell
- **Entry Price:** 1.07956
- **Take Profit:** 1.0750
- **Stop Loss:** 1.0810
- **Deadline Date:** 2024-05-30T18:00:00Z

### Four-Hours Timeframe
- **Patterns Detected:**
  - **Pattern ID:** 3
    - **Pattern Name:** Bearish Engulfing
    - **Confidence Percentage:** 80
- **Action:** Sell
- **Entry Price:** 1.07958
- **Take Profit:** 1.0750
- **Stop Loss:** 1.0810
- **Deadline Date:** 2024-05-30T18:00:00Z

### Third Analysis Result
### Normal Timeframe
- **Patterns Detected:**
  - **Pattern ID:** 1
    - **Pattern Name:** Bullish Engulfing
    - **Confidence Percentage:** 75
- **Action:** Buy
- **Entry Price:** 1.0798
- **Take Profit:** 1.0850
- **Stop Loss:** 1.0770
- **Deadline Date:** 2024-06-02T18:00:00Z

### One-Hour Timeframe
- **Patterns Detected:**
  - **Pattern ID:** 2
    - **Pattern Name:** Inverse Head and Shoulders
    - **Confidence Percentage:** 80
- **Action:** Buy
- **Entry Price:** 1.0796
- **Take Profit:** 1.0840
- **Stop Loss:** 1.0760
- **Deadline Date:** 2024-05-31T18:00:00Z

### Four-Hours Timeframe
- **Patterns Detected:**
  - **Pattern ID:** 3
    - **Pattern Name:** Bullish Divergence
    - **Confidence Percentage:** 78
- **Action:** Buy
- **Entry Price:** 1.0795
- **Take Profit:** 1.0835
- **Stop Loss:** 1.0765
- **Deadline Date:** 2024-06-01T18:00:00Z

{
    "timeframe": "normal",
    "patterns": [
        {
            "id": 1,
            "pattern_detected": 1,
            "pattern_name": "Double Top",
            "confidence_percentage": 90
        }
    ],
    "action": "sell",
    "entry_price": 1.07956,
    "take_profit": 1.0750,
    "stop_loss": 1.0810,
    "deadline_date": "2024-05-30T18:00:00Z"
}


In [17]:
import json

# Function to place an order based on the highest confidence pattern
def place_high_confidence_order():
    # Highest confidence pattern in the normal (5-minute) timeframe
    order_details = {
        "id": 1,
        "pattern_name": "Double Top",
        "pattern_type": "Reversal",
        "confidence_percentage": 90,
        "entry_point": "1.07956",
        "take_profit": "1.0750",
        "stop_loss": "1.0810",
        "best_exit_time": "2024-05-30T18:00:00Z",
        "order_id": "ORD123456"
    }

    # Log the order details
    with open('orders_log.json', 'a') as log_file:
        json.dump(order_details, log_file, indent=4)
        log_file.write('\n')

    print(f"Placing order: {order_details}")

# Call the function to place the order
place_high_confidence_order()


Placing order: {'id': 1, 'pattern_name': 'Double Top', 'pattern_type': 'Reversal', 'confidence_percentage': 90, 'entry_point': '1.07956', 'take_profit': '1.0750', 'stop_loss': '1.0810', 'best_exit_time': '2024-05-30T18:00:00Z', 'order_id': 'ORD123456'}


In [22]:
import v20
import json
import os
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

# OANDA API configuration using environment variables
api = v20.Context(
    hostname='api-fxpractice.oanda.com',
    port=443,
    ssl=True,
    token=os.getenv('OANDA_API_TOKEN'),
    datetime_format='RFC3339'
)

# Function to get the status of an order
def check_order_status(order_id):
    try:
        response = api.order.order.get(
            accountID=os.getenv('OANDA_ACCOUNT_ID'),
            orderID=order_id
        )
        if response.status == 200:
            order_status = response.body.get('order', {}).get('state', 'Unknown')
            print(f"Order ID {order_id} status: {order_status}")
            return order_status
        else:
            print(f"Failed to retrieve status for Order ID {order_id}")
            return None
    except Exception as e:
        print(f"An error occurred while checking order status: {e}")
        return None

# Load the latest order ID from the log file
def get_latest_order_id():
    try:
        with open('orders_log.json', 'r') as log_file:
            orders = log_file.readlines()
            if orders:
                latest_order = json.loads(orders[-1])
                return latest_order.get('order_id', None)
            else:
                print("No orders found in the log file.")
                return None
    except Exception as e:
        print(f"An error occurred while reading the log file: {e}")
        return None

# Check the status of the latest order
latest_order_id = "ORD123456"
if latest_order_id:
    check_order_status('ORD123456')
else:
    print("No order ID available to check the status.")


An error occurred while checking order status: 'EntitySpec' object has no attribute 'order'


In [ ]:
import v20
import json
import os
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

# OANDA API configuration using environment variables
api = v20.Context(
    hostname='api-fxpractice.oanda.com',
    port=443,
    ssl=True,
    token=os.getenv('OANDA_API_TOKEN'),
    datetime_format='RFC3339'
)

# Function to get the status of an order
def check_order_status(order_id):
    try:
        response = api.order.order.get(
            accountID=os.getenv('OANDA_ACCOUNT_ID'),
            orderID=order_id
        )
        if response.status == 200:
            order_status = response.body.get('order', {}).get('state', 'Unknown')
            print(f"Order ID {order_id} status: {order_status}")
            return order_status
        else:
            print(f"Failed to retrieve status for Order ID {order_id}")
            return None
    except Exception as e:
        print(f"An error occurred while checking order status: {e}")
        return None

# Load the latest order ID from the log file
def get_latest_order_id():
    try:
        with open('orders_log.json', 'r') as log_file:
            orders = log_file.readlines()
            if orders:
                latest_order = json.loads(orders[-1])
                return latest_order.get('order_id', None)
            else:
                print("No orders found in the log file.")
                return None
    except Exception as e:
        print(f"An error occurred while reading the log file: {e}")
        return None

# Check the status of the latest order
latest_order_id = "ORD123456"
if latest_order_id:
    check_order_status('ORD123456')
else:
    print("No order ID available to check the status.")


An error occurred while checking order status: 'EntitySpec' object has no attribute 'order'
